In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

import os
import sys
import warnings

# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath("./../src"))

import matplotlib.pyplot as plt
import matplotlib.colors

import numpy as np
import seaborn as sns

import gan_regression

import importlib
importlib.reload(gan_regression)

plt.rcdefaults()
warnings.filterwarnings("ignore")

%matplotlib qt

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [2]:
VERSION = "v1"
MODEL_TYPE = "LOWER_BAND"
pdata_folder = os.path.abspath(r"./../processed_data/chorus_neural_network/")
STAGE_4_folder = os.path.join(pdata_folder, "models", VERSION)
CONJUNCTIONS_REFS = np.load(
    file=os.path.join(STAGE_4_folder, f"MODEL_READY_dataset_{VERSION}_{MODEL_TYPE}.npz")
)

X_train = CONJUNCTIONS_REFS["FEATURES"]
y_train = CONJUNCTIONS_REFS["LABELS"].flatten()
day_train = CONJUNCTIONS_REFS["TRAINING_DAY_IDS"].flatten()

X_valid = CONJUNCTIONS_REFS["VALIDATION_FEATURES"]
y_valid = CONJUNCTIONS_REFS["VALIDATION_LABELS"].flatten()

CONJUNCTIONS_REFS.close()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Dev\\Research\\Research_Tools\\processed_data\\chorus_neural_network\\models\\v1\\MODEL_READY_dataset_v1_LOWER_BAND.npz'

In [ ]:
print(f"Training set shape: {X_train.shape, y_train.shape}")
print(f"Validation set shape: {X_valid.shape, y_valid.shape}")

ax0 = sns.displot(y_train, log_scale=True)
ax0.set(ylabel='N', xlabel='Chorus Power (pT^2)', title='Training')

In [27]:
# Initialize and train model
model = gan_regression.GANRegression(input_dim=6,
                                     hidden_dim=128,
                                     g_lr=2e-7,
                                     d_lr=2e-7,
                                     epochs_per_output=1)

avg_d_losses, avg_numeric_losses, avg_g_losses = model.train(X_train, y_train, epochs=20, batch_size=32)

using device : cuda
Epoch [1 / 20] 
D Loss: (48.4190 +/- 1.5191) N Loss: (15073.6783 +/- 25858.5637) G Loss: (0.0127 +/- 0.0146)

Epoch [2 / 20] 
D Loss: (48.1096 +/- 1.6624) N Loss: (15570.3511 +/- 25454.7962) G Loss: (0.0136 +/- 0.0158)

Epoch [3 / 20] 
D Loss: (44.2774 +/- 7.6452) N Loss: (15311.2835 +/- 24757.1216) G Loss: (0.0500 +/- 0.4273)

Epoch [4 / 20] 
D Loss: (0.1555 +/- 0.2124) N Loss: (10947.1443 +/- 18169.5769) G Loss: (14.8552 +/- 4.7658)

Epoch [5 / 20] 
D Loss: (0.1426 +/- 0.0833) N Loss: (5075.7188 +/- 9342.7172) G Loss: (12.2447 +/- 4.7640)

Epoch [6 / 20] 
D Loss: (3.1289 +/- 2.7516) N Loss: (1491.1475 +/- 2355.9409) G Loss: (2.9852 +/- 2.8579)

Epoch [7 / 20] 
D Loss: (9.4187 +/- 2.8750) N Loss: (4872.7706 +/- 7837.1282) G Loss: (2.3769 +/- 1.9324)

Epoch [8 / 20] 
D Loss: (13.9429 +/- 2.9067) N Loss: (9373.7020 +/- 15311.3544) G Loss: (2.7168 +/- 1.9944)

Epoch [9 / 20] 
D Loss: (15.5220 +/- 2.8482) N Loss: (14080.6860 +/- 23078.6026) G Loss: (2.9600 +/- 2.0361)


In [22]:
threshold = 0.0

y_train_g = model.GENERATE(X_train)

y_train_d = model.DISCRIM(X_train, y_train_g)

passes_discriminator_test = y_train_d >= threshold
plt.scatter(x=np.sqrt(y_train[passes_discriminator_test]), y=np.sqrt(y_train_g[passes_discriminator_test]), color="black", s=1.0)

plt.xscale("log")
plt.yscale("log")
plt.xlim(1e-1, 1e3)
plt.ylim(1e-1, 1e3)

(0.1, 1000.0)

In [12]:
y_valid_g = model.GENERATE(X_valid)
y_valid_d = model.DISCRIM(X_valid, y_valid_g)

passes_discriminator_test = y_valid_d >= threshold
plt.scatter(x=y_valid[passes_discriminator_test], y=y_valid_g[passes_discriminator_test], color="red", s=1.0)
plt.xscale("log")
plt.yscale("log")
plt.xlim(1e-1, 1e3)


plt.ylim(1e-1, 1e3)

(0.1, 1000.0)

In [ ]:
print(y_train_g)
print(np.log10(y_train_g))

In [11]:
not_negative = y_train_g > 0
passes_discriminator_test = y_train_d >= threshold
passes_both_filters = not_negative & passes_discriminator_test

a, b = np.polyfit(np.log10(y_train[passes_both_filters]), np.log10(y_train_g[passes_both_filters]), 1)

x_fit = np.logspace(-1, 4, 1000)

# Create the line of best fit
y_fit = a * np.log10(x_fit) + b

# Plot the line of best fit
plt.plot(x_fit, 10**y_fit, color='red')
plt.plot(x_fit, x_fit, color='green')


print(a)


0.11282530245189473


In [ ]:
not_negative = y_valid_g > 0
passes_discriminator_test = y_valid_d >= threshold
passes_both_filters = not_negative & passes_discriminator_test

a, b = np.polyfit(np.log10(y_valid[passes_both_filters]), np.log10(y_valid_g[passes_both_filters]), 1)

x_fit = np.logspace(-1, 4, 1000)

# Create the line of best fit
y_fit = a * np.log10(x_fit) + b

# Plot the line of best fit
plt.plot(x_fit, 10**y_fit, color='red')
plt.plot(x_fit, x_fit, color='green')


print(a)